In [60]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [62]:
IMAGES_DIR = '../augm_img'  # Путь к датасету с изображениями
MASKS_DIR = '../augm_msk'  # Путь к датасету с масками
OUTPUT_DIR = 'train_data'  # Путь к выходной директории
TRAIN_SIZE = 0.9  # Процент обучающей выборки

In [7]:
# Создание путей к суммарному датасету
os.makedirs(os.path.join(OUTPUT_DIR, 'images/all'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/all'), exist_ok=True)

In [70]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return
    
    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [71]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [72]:
# Копирование изображений и масок в соответствующие папки
for img in image_files:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/all', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/all')

In [75]:
# Разделение на train/val
files = os.listdir(os.path.join(OUTPUT_DIR, 'images/all'))
train_images, val_images = train_test_split(files, train_size=TRAIN_SIZE, random_state=42)
len(train_images), len(val_images), len(files)

(7230, 804, 8034)

In [76]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)
# Копирование изображений и масок в соответствующие папки
for img in train_images:
    shutil.copy(os.path.join(OUTPUT_DIR, 'images/all', img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.png', '.txt').replace('.jpg', '.txt')
    shutil.copy(os.path.join(OUTPUT_DIR, 'labels/all', mask_name), os.path.join(OUTPUT_DIR, 'labels/train', mask_name))
for img in val_images:
    shutil.copy(os.path.join(OUTPUT_DIR, 'images/all', img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.png', '.txt').replace('.jpg', '.txt')
    shutil.copy(os.path.join(OUTPUT_DIR, 'labels/all', mask_name), os.path.join(OUTPUT_DIR, 'labels/val', mask_name))

In [77]:
data_yaml_content = f"""
path: train_data/images
train: train
val: val

nc: 1
names: ['contaminated']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [78]:
model = YOLO("yolo11n-seg.pt")

In [82]:
train_results = model.train(
    data="./data.yaml",  # path to dataset YAML
    batch=32,
    epochs=32,  # number of training epochs
    imgsz=640,  # training image size
    device="cuda",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

New https://pypi.org/project/ultralytics/8.3.47 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.13  Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=segment, mode=train, model=best.pt, data=./data.yaml, epochs=32, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning C:\Users\VLAD\Downloads\dirtybusiness\train_dataset\baseline\train_data\labels\train.cache... 7230 imag

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\VLAD\Downloads\dirtybusiness\train_dataset\baseline\train_data\labels\val.cache... 804 images, 3


Plotting labels to runs\segment\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train10
Starting training for 32 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      32/32      6.19G     0.6469      1.332     0.6177      1.004         52        640: 100%|██████████| 226/226 [01:
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

                   all        804       1572      0.934      0.784      0.853      0.715      0.934      0.784      0.842      0.646



32 epochs completed in 0.734 hours.
Optimizer stripped from runs\segment\train10\weights\last.pt, 6.0MB
Optimizer stripped from runs\segment\train10\weights\best.pt, 6.0MB

Validating runs\segment\train10\weights\best.pt...
Ultralytics 8.3.13  Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all        804       1572      0.935      0.782      0.853      0.715      0.935      0.784      0.842      0.646
Speed: 0.3ms preprocess, 1.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\segment\train10


In [92]:
model = YOLO('best.pt')